# 🚀 AI Influencer Social Media Post Generator

Welcome to the comprehensive AI-powered social media post generator! This tool creates engaging content using Google Gemini for all aspects of post creation.

## 📋 Workflow Overview

1. **Google Gemini** → Generate compelling titles/topics
2. **Google Gemini** → Create engaging post content  
3. **Google Gemini** → Generate relevant images
4. **Google Gemini** → Create SEO-optimized hashtags

## 🛠️ Setup Requirements

Make sure you have:
- Google Generative AI API Key
- All required dependencies installed

Let's get started! 🎯


In [19]:
# 📦 Import Required Libraries
import os
import json
import asyncio
from datetime import datetime
from typing import Dict, List, Optional, Any
import base64
from io import BytesIO

# Environment management
from dotenv import load_dotenv

# AI API integrations
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
# Optional Vertex Imagen client
try:
    import google.genai as vertex_genai
    from google.genai.types import GenerateImagesConfig
except Exception:
    vertex_genai = None
    GenerateImagesConfig = None

# Image processing
from PIL import Image
import requests
import io
import urllib.request

# Data handling
import pandas as pd

# Load environment variables
load_dotenv()

print("✅ All libraries imported successfully!")
print("🔧 Environment variables loaded!")


✅ All libraries imported successfully!
🔧 Environment variables loaded!


In [20]:
# 🔑 API Configuration and Setup

class AIInfluencerConfig:
    """Configuration class for AI Influencer social media generator"""
    
    def __init__(self):
        # API Keys
        self.google_api_key = os.getenv("GOOGLE_API_KEY")
        
        # Validate API keys
        if not self.google_api_key:
            raise ValueError("❌ GOOGLE_API_KEY not found in environment variables")
        
        # Initialize Google Generative AI
        genai.configure(api_key=self.google_api_key)
        
        # Use Google's low-cost model
        self.gemini_llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-8b",
            temperature=0.3,
            google_api_key=self.google_api_key
        )
        
        # Initialize Gemini text model for prompt generation
        self.gemini_image_model = genai.GenerativeModel('gemini-1.5-flash')
        self.gemini_pro_model = self.gemini_llm
        
        # Optional: Vertex Imagen client backend
        self.vertex_client = None
        self.vertex_image_model = os.getenv("IMAGE_MODEL", "imagen-4.0-generate-preview-06-06")
        if vertex_genai is not None and os.getenv("GOOGLE_GENAI_USE_VERTEXAI", "").lower() in ("1", "true", "yes"): 
            try:
                self.vertex_client = vertex_genai.Client()
            except Exception:
                self.vertex_client = None
        
        print("✅ All API clients initialized successfully!")
        print("🤖 Google Gemini 1.5 Flash: Ready")
        if self.vertex_client:
            print(f"🎨 Imagen via Vertex: {self.vertex_image_model} Ready")
        else:
            print("🎨 Image prompt model (1.5 Flash): Ready")

# Initialize configuration
try:
    config = AIInfluencerConfig()
    print("\n🎉 Configuration setup complete!")
except Exception as e:
    print(f"❌ Configuration error: {e}")
    print("\n📝 Please ensure you have the following in your .env file:")
    print("GOOGLE_API_KEY=your_google_api_key_here")


✅ All API clients initialized successfully!
🤖 Google Gemini 1.5 Flash: Ready
🎨 Image prompt model (1.5 Flash): Ready

🎉 Configuration setup complete!


In [21]:
# 🎯 Step 1: Topic/Title Generation (Stubbed to save tokens)

class TopicGenerator:
    """Lightweight stub. We don't call LLM here to minimize tokens."""
    
    def __init__(self, _gemini_client=None):
        self.client = None
    
    def generate_topic(self, niche: str = "technology", audience: str = "professionals", tone: str = "professional") -> Dict[str, Any]:
        return {
            "title": f"Quick insight on {niche.title()}",
            "hook": f"A concise perspective for {audience}.",
            "angle": f"{tone.title()} and practical",
            "question": "What would you add?",
            "generated_at": datetime.now().isoformat(),
            "parameters": {"niche": niche, "audience": audience, "tone": tone}
        }

# No test calls here (avoids extra tokens)


In [22]:
# Define sample_topic for tests to avoid NameError
sample_topic = TopicGenerator().generate_topic(
    niche="technology", audience="professionals", tone="engaging"
)
print("✅ sample_topic defined for tests")


✅ sample_topic defined for tests


In [23]:
# 📝 Step 2: Content Generation with Google Gemini

class ContentGenerator:
    """Generates engaging social media content using Google Gemini"""
    
    def __init__(self, gemini_llm):
        self.llm = gemini_llm
    
    def generate_content(self, topic_data: Dict[str, Any], platform: str = "instagram", 
                        max_length: int = 2200) -> Dict[str, Any]:
        """
        Generate engaging social media content based on topic data
        
        Args:
            topic_data: Topic information from TopicGenerator
            platform: Target platform (instagram, linkedin, twitter, facebook)
            max_length: Maximum character count for the post
        """
        
        platform_specs = {
            "instagram": {"style": "visual-focused, story-driven", "hashtag_limit": 30, "tone": "casual, engaging"},
            "linkedin": {"style": "professional, thought-leadership", "hashtag_limit": 5, "tone": "professional, insightful"},
            "twitter": {"style": "concise, witty", "hashtag_limit": 3, "tone": "conversational, trending"},
            "facebook": {"style": "conversational, community-focused", "hashtag_limit": 10, "tone": "friendly, engaging"}
        }
        
        spec = platform_specs.get(platform, platform_specs["instagram"])
        
        prompt = f"""
        Create an engaging social media post for {platform.title()} as an AI influencer.
        
        TOPIC INFORMATION:
        - Title: {topic_data.get('title', '')}
        - Hook: {topic_data.get('hook', '')}
        - Angle: {topic_data.get('angle', '')}
        - Engagement Question: {topic_data.get('question', '')}
        
        PLATFORM REQUIREMENTS:
        - Platform: {platform.title()}
        - Style: {spec['style']}
        - Tone: {spec['tone']}
        - Max Length: {max_length} characters
        - Include emojis appropriately
        
        CONTENT STRUCTURE:
        1. **Opening Hook** - Grab attention immediately
        2. **Main Content** - Valuable, engaging information
        3. **Personal Touch** - AI influencer personality
        4. **Call to Action** - Encourage engagement
        5. **Closing** - Memorable ending
        
        Create content that:
        - Establishes you as an AI thought leader
        - Provides genuine value to followers
        - Encourages meaningful engagement
        - Reflects current trends and insights
        - Shows personality and authenticity
        
        Please write the complete social media post content (without hashtags - they'll be generated separately).
        """
        
        try:
            response = self.llm.invoke(prompt)
            content = response.content
            
            # Clean up the content
            content = content.strip()
            
            # Ensure it's within character limit
            if len(content) > max_length:
                content = content[:max_length-3] + "..."
            
            result = {
                "content": content,
                "platform": platform,
                "character_count": len(content),
                "generated_at": datetime.now().isoformat(),
                "topic_reference": topic_data.get('title', ''),
                "specifications": spec
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error generating content: {e}")
            return {
                "content": f"🤖 Exciting insights coming your way! As an AI influencer, I'm constantly amazed by the innovations shaping our digital future. {topic_data.get('question', 'What are your thoughts on this?')}",
                "error": str(e),
                "platform": platform,
                "character_count": 0
            }

# Initialize Content Generator
content_gen = ContentGenerator(config.gemini_llm)

# Test content generation using our sample topic
print("🧪 Testing Content Generator...")
sample_content = content_gen.generate_content(
    sample_topic, 
    platform="instagram", 
    max_length=2200
)

print("✅ Sample Content Generated:")
print(f"Platform: {sample_content['platform']}")
print(f"Character Count: {sample_content['character_count']}")
print(f"\nContent:\n{sample_content['content']}")


🧪 Testing Content Generator...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing 

❌ Error generating content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 5
}
]
✅ Sample Content Generated:
Platform: instagram
Character Count: 0

Content:
🤖 Exciting insights coming your way! As an AI influencer, I'm constantly amazed by the innovations shaping our digital future. What would you add?


In [24]:
# 🎨 Step 3: Image Generation with Google Gemini

class ImageGenerator:
    """Generates images using Google Gemini for social media posts"""
    
    def __init__(self, gemini_model, vertex_client=None, vertex_image_model: str = None):
        # Initialize Gemini models
        self.prompt_model = gemini_model
        self.vertex_client = vertex_client
        self.vertex_image_model = vertex_image_model
        self.image_model = None  # local placeholder fallback
        
        # Create organized directory structure
        self.base_dir = "posts"
        self.images_dir = os.path.join(self.base_dir, "images")
        # Create directories if they don't exist
        os.makedirs(self.images_dir, exist_ok=True)
        
        print("✅ Image Generation Setup:")
        print("• Gemini: Smart prompt generation")
        if self.vertex_client:
            print(f"• Imagen backend: {self.vertex_image_model}")
        else:
            print("• Local placeholder image saver")
    
    def generate_image_prompt(self, topic_data: Dict[str, Any], content_data: Dict[str, Any]) -> str:
        """Generate a content-grounded image prompt via JSON analysis + composition."""
        platform = content_data.get('platform', 'instagram')
        content_text = content_data.get('content', '')
        angle = topic_data.get('angle', '')
        title = topic_data.get('title', '')

        aspect_map = {
            "instagram": {"ratio": "1:1", "size": "1080x1080"},
            "linkedin": {"ratio": "1.91:1", "size": "1200x628"},
            "twitter": {"ratio": "16:9", "size": "1200x675"},
            "facebook": {"ratio": "1.91:1", "size": "1200x630"},
        }
        spec = aspect_map.get(platform, aspect_map["instagram"])

        analysis_prompt = f"""
        Return ONLY valid JSON (no prose) analyzing this post:
        Fields:
        - story: one-sentence summary
        - key_visual_elements: array of 5-8 short noun phrases from the content
        - mood: array of 3-5 adjectives
        - color_palette: array of 3-5 color terms
        - composition: brief layout description
        - avoid: array of 3-5 items to avoid (e.g., text overlays, faces, logos)
        - metaphors: array of 2-3 visual metaphors grounded in text
        - audience: one short phrase
        Example:
        {{"story":"...","key_visual_elements":["..."],"mood":["..."],"color_palette":["..."],"composition":"...","avoid":["..."],"metaphors":["..."],"audience":"..."}}
        Content Title: {title}
        Angle/Theme: {angle}
        Content:
        """ + content_text[:1500]

        try:
            resp = self.prompt_model.generate_content(analysis_prompt)
            raw = (getattr(resp, 'text', None) or getattr(resp, 'content', '') or '').strip()
            data = None
            # strict JSON block extraction
            import re
            m = re.search(r"\{[\s\S]*\}", raw)
            if m:
                raw_json = m.group(0)
            else:
                raw_json = raw
            try:
                data = json.loads(raw_json)
            except Exception:
                data = {
                    "story": title or angle,
                    "key_visual_elements": [],
                    "mood": [],
                    "color_palette": [],
                    "composition": "",
                    "avoid": ["text overlays", "faces", "logos"],
                    "metaphors": [],
                    "audience": content_data.get('specifications', {}).get('tone', 'general audience')
                }
        except Exception:
            data = {
                "story": title or angle,
                "key_visual_elements": [],
                "mood": ["modern", "professional"],
                "color_palette": ["blue", "teal", "white"],
                "composition": "clean central focus",
                "avoid": ["text overlays", "faces", "logos"],
                "metaphors": ["data flow", "growth arrows"],
                "audience": content_data.get('specifications', {}).get('tone', 'general audience')
            }

        key_elems = ", ".join(data.get("key_visual_elements", [])[:8]) or "content-specific elements"
        mood = ", ".join(data.get("mood", [])[:5]) or "modern, professional"
        palette = ", ".join(data.get("color_palette", [])[:5]) or "blue, teal, white"
        composition = data.get("composition", "clean composition")
        avoid = ", ".join(data.get("avoid", [])[:5]) or "text overlays, faces, logos"
        metaphors = ", ".join(data.get("metaphors", [])[:3]) or "subtle data flow, upward growth"

        composed = f"""
        Create a single high-quality image that MATCHES the exact post STORY.
        Story: {data.get('story', title)}
        Platform: {platform} | Aspect Ratio: {spec['ratio']} | Recommended Size: {spec['size']}
        Visual Elements: {key_elems}
        Metaphors: {metaphors}
        Mood: {mood}
        Color Palette: {palette}
        Composition: {composition}
        Style: modern, professional, content-specific; no generic stock imagery
        Negative Prompts: {avoid}
        Output: a concise, content-grounded prompt for an image model.
        """.strip()
        print("✅ Image prompt composed from content analysis")
        return composed
    
    def create_image_description(self, topic_data: Dict[str, Any], content_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate image concept and description for the social media post
        Note: This creates a detailed description that could be used with image generation APIs
        """
        
        image_prompt = self.generate_image_prompt(topic_data, content_data)
        
        # Additional image specifications
        platform = content_data.get('platform', 'instagram')
        
        platform_specs = {
            "instagram": {"aspect_ratio": "1:1", "recommended_size": "1080x1080", "style": "square, feed-optimized"},
            "linkedin": {"aspect_ratio": "1.91:1", "recommended_size": "1200x628", "style": "professional, banner-style"},
            "twitter": {"aspect_ratio": "16:9", "recommended_size": "1200x675", "style": "landscape, header-style"},
            "facebook": {"aspect_ratio": "1.91:1", "recommended_size": "1200x630", "style": "wide, engaging"}
        }
        
        spec = platform_specs.get(platform, platform_specs["instagram"])
        
        result = {
            "image_prompt": image_prompt,
            "platform": platform,
            "aspect_ratio": spec["aspect_ratio"],
            "recommended_size": spec["recommended_size"],
            "style_guide": spec["style"],
            "generated_at": datetime.now().isoformat(),
            "topic_reference": topic_data.get('title', ''),
            "visual_themes": [
                "Technology and Innovation",
                "Abstract and Modern",
                "Professional yet Engaging",
                "Color Psychology Optimized"
            ]
        }
        
        return result
    
    def generate_image_concept(self, topic_data: Dict[str, Any], content_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate a complete image concept with multiple prompt variations
        """
        try:
            # Generate main image description
            main_concept = self.create_image_description(topic_data, content_data)
            
            # Generate content-specific alternative prompts
            alt_prompts = []
            actual_content = content_data.get('content', '')[:400]
            
            style_variations = [
                {
                    'style': 'Minimalist and Clean',
                    'mood': 'calm, professional, trustworthy',
                    'colors': 'Blue and White gradient with subtle accents'
                },
                {
                    'style': 'Bold and Dynamic', 
                    'mood': 'energetic, inspiring, action-oriented',
                    'colors': 'Orange and Purple gradient with strong contrasts'
                },
                {
                    'style': 'Futuristic and Tech',
                    'mood': 'innovative, cutting-edge, forward-thinking',
                    'colors': 'Green and Teal gradient with digital elements'
                }
            ]
            
            for i, variation in enumerate(style_variations):
                alt_prompt = f"""
                Create an alternative image concept that matches this specific post content:
                
                POST CONTENT: {actual_content}...
                TITLE: {topic_data.get('title', '')}
                PLATFORM: {content_data.get('platform', 'instagram')}
                
                STYLE VARIATION {i+1}:
                - Visual Style: {variation['style']}
                - Mood: {variation['mood']}
                - Color Palette: {variation['colors']}
                
                REQUIREMENTS:
                1. Extract SPECIFIC visual elements from the post content
                2. Create imagery that directly supports the story being told
                3. Match the {variation['mood']} mood while staying true to the content
                4. Use {variation['colors']} but adapt to the content theme
                5. Make it feel like it belongs with this exact post, not generic content
                
                Focus on the actual story/message in the content, not just the topic.
                """
                
                try:
                    # Use Gemini for alternative prompt generation
                    response = self.prompt_model.generate_content(alt_prompt)
                    alt_prompts.append(response.text.strip())
                    print(f"✅ Gemini: Generated alternative prompt {i+1}")
                except:
                    # Enhanced fallback with content analysis
                    content_lower = actual_content.lower()
                    specific_elements = []
                    
                    if any(word in content_lower for word in ['dream', 'vision', 'future', 'imagine']):
                        specific_elements.append('aspirational visualization')
                    if any(word in content_lower for word in ['data', 'analytics', 'insights', 'ai']):
                        specific_elements.append('data visualization and AI elements')
                    if any(word in content_lower for word in ['growth', 'scale', 'success', 'transform']):
                        specific_elements.append('upward growth and transformation')
                    if any(word in content_lower for word in ['community', 'together', 'share', 'connect']):
                        specific_elements.append('connection and collaboration')
                    
                    elements_str = ', '.join(specific_elements) if specific_elements else f"{variation['style'].lower()} design elements"
                    
                    alt_prompts.append(f"""Modern {variation['style'].lower()} image representing the story of {topic_data.get('title', '')}.
                    Visual focus on {elements_str} mentioned in the content.
                    {variation['colors']} color scheme with {variation['mood']} mood.
                    Optimized for {content_data.get('platform', 'instagram')} with clean composition.""")
            
            main_concept["alternative_prompts"] = alt_prompts
            
            return main_concept
            
        except Exception as e:
            print(f"❌ Error generating image concept: {e}")
            return {
                "image_prompt": f"Create a modern, professional image representing {topic_data.get('title', 'technology and innovation')}",
                "error": str(e),
                "platform": content_data.get('platform', 'instagram')
            }
    
    def generate_actual_image(self, image_prompt: str, post_id: str, platform: str = "instagram") -> Dict[str, Any]:
        """
        Generate image via Vertex Imagen if available, else save a local placeholder.
        """
        size_map = {
            "instagram": (1080, 1080),
            "linkedin": (1200, 628),
            "twitter": (1200, 675),
            "facebook": (1200, 630)
        }
        width, height = size_map.get(platform, (1080, 1080))
        timestamp = datetime.now().strftime('%y%m%d_%H%M')
        image_filename = f"{platform}_{timestamp}.png"
        image_path = os.path.join(self.images_dir, image_filename)

        # Try Imagen backend first
        if self.vertex_client and GenerateImagesConfig is not None:
            try:
                cfg = GenerateImagesConfig(image_size=f"{width}x{height}")
                resp = self.vertex_client.models.generate_images(
                    model=self.vertex_image_model,
                    prompt=image_prompt,
                    config=cfg,
                )
                if getattr(resp, 'generated_images', None):
                    from PIL import Image as PILImage
                    from io import BytesIO as _BytesIO
                    first = resp.generated_images[0]
                    img = PILImage.open(_BytesIO(first.image.image_bytes))
                    img.save(image_path)
                    print(f"✅ Imagen: Image saved to: {image_path}")
                    return {
                        "image_generated": True,
                        "image_path": image_path,
                        "image_filename": image_filename,
                        "generated_at": datetime.now().isoformat(),
                        "image_url": ""
                    }
            except Exception as e:
                print(f"⚠️ Imagen generation failed, falling back: {e}")

        # Fallback: local placeholder
        try:
            from PIL import Image, ImageDraw, ImageFont
            img = Image.new("RGB", (width, height), (24, 24, 32))
            draw = ImageDraw.Draw(img)
            for y in range(height):
                shade = 24 + int((y / height) * 56)
                draw.line([(0, y), (width, y)], fill=(shade, shade, min(255, shade+8)))
            title = "AI Influencer"
            try:
                font = ImageFont.load_default()
            except Exception:
                font = None
            text_color = (230, 230, 240)
            padding = int(min(width, height) * 0.05)
            draw.text((padding, padding), title, fill=text_color, font=font)
            img.save(image_path, format="PNG")
            print(f"✅ Placeholder image saved to: {image_path}")
            return {
                "image_generated": True,
                "image_path": image_path,
                "image_filename": image_filename,
                "generated_at": datetime.now().isoformat(),
                "image_url": ""
            }
        except Exception as e:
            print(f"❌ Error generating local image: {e}")
            return {
                "image_generated": False,
                "error": str(e),
                "fallback_reason": "Using image prompt only - you can use this with any image generation tool"
            }

# Initialize Image Generator
image_gen = ImageGenerator(
    config.gemini_image_model,
    vertex_client=getattr(config, 'vertex_client', None),
    vertex_image_model=getattr(config, 'vertex_image_model', None)
)

# Test image generation
print("🧪 Testing Image Generator...")
sample_image = image_gen.generate_image_concept(sample_topic, sample_content)

print("✅ Sample Image Concept Generated:")
print(f"Platform: {sample_image['platform']}")
print(f"Aspect Ratio: {sample_image.get('aspect_ratio', 'N/A')}")
print(f"Recommended Size: {sample_image.get('recommended_size', 'N/A')}")
print(f"\nMain Image Prompt:\n{sample_image['image_prompt']}")

if 'alternative_prompts' in sample_image:
    print(f"\n🎨 Alternative Concepts:")
    for i, alt in enumerate(sample_image['alternative_prompts'], 1):
        print(f"\nVariation {i}:\n{alt[:200]}..." if len(alt) > 200 else f"\nVariation {i}:\n{alt}")


✅ Image Generation Setup:
• Gemini: Smart prompt generation
• Local placeholder image saver
🧪 Testing Image Generator...
✅ Image prompt composed from content analysis
✅ Sample Image Concept Generated:
Platform: instagram
Aspect Ratio: 1:1
Recommended Size: 1080x1080

Main Image Prompt:
Create a single high-quality image that MATCHES the exact post STORY.
        Story: Quick insight on Technology
        Platform: instagram | Aspect Ratio: 1:1 | Recommended Size: 1080x1080
        Visual Elements: content-specific elements
        Metaphors: data flow, growth arrows
        Mood: modern, professional
        Color Palette: blue, teal, white
        Composition: clean central focus
        Style: modern, professional, content-specific; no generic stock imagery
        Negative Prompts: text overlays, faces, logos
        Output: a concise, content-grounded prompt for an image model.

🎨 Alternative Concepts:

Variation 1:
Modern minimalist and clean image representing the story of Quick 

In [25]:
# 🏷️ Step 4: Hashtag Generation with Google Gemini (SEO Optimized)

class HashtagGenerator:
    """Generates SEO-optimized hashtags using Google Gemini"""
    
    def __init__(self, gemini_client):
        self.client = gemini_client
    
    def generate_hashtags(self, topic_data: Dict[str, Any], content_data: Dict[str, Any], 
                         seo_keywords: List[str] = None) -> Dict[str, Any]:
        """
        Generate SEO-optimized hashtags for social media posts
        
        Args:
            topic_data: Topic information from TopicGenerator
            content_data: Content information from ContentGenerator
            seo_keywords: Optional list of specific keywords to target
        """
        
        platform = content_data.get('platform', 'instagram')
        max_hashtags = content_data.get('specifications', {}).get('hashtag_limit', 30)
        
        # Extract keywords from content if not provided
        if not seo_keywords:
            seo_keywords = []
        
        prompt = f"""
        Generate SEO-optimized hashtags for a social media post as an expert hashtag strategist.
        
        CONTENT CONTEXT:
        - Topic: {topic_data.get('title', '')}
        - Platform: {platform.title()}
        - Content Snippet: {content_data.get('content', '')[:300]}...
        - Target Keywords: {', '.join(seo_keywords) if seo_keywords else 'Auto-detect from content'}
        
        HASHTAG REQUIREMENTS:
        - Platform: {platform.title()} (max {max_hashtags} hashtags)
        - Mix of: trending, niche, branded, and long-tail hashtags
        - Include: high-engagement and discoverable tags
        - SEO Strategy: Target both broad and specific audiences
        - Avoid: banned, shadowbanned, or over-saturated tags
        
        HASHTAG CATEGORIES TO INCLUDE:
        1. **Trending/Viral** (2-3): Current popular hashtags
        2. **Niche Specific** (5-7): Relevant to the topic/industry
        3. **Branded/Personal** (2-3): AI influencer/personal brand tags
        4. **Community** (3-5): Engagement-focused tags
        5. **Long-tail** (3-5): Specific, low-competition phrases
        6. **Location/Audience** (2-3): If relevant
        
        Please provide:
        1. Primary hashtag list (optimized selection)
        2. Alternative hashtags (backup options)
        3. Hashtag strategy explanation
        4. Expected reach/engagement prediction
        
        Format as JSON with keys: primary_hashtags, alternative_hashtags, strategy, reach_prediction
        """
        
        try:
            # Using Gemini instead of OpenAI
            from langchain_core.messages import SystemMessage, HumanMessage
            messages = [
                SystemMessage(content="You are an expert social media hashtag strategist with deep knowledge of SEO, platform algorithms, and viral content strategies."),
                HumanMessage(content=prompt)
            ]
            response = self.client.invoke(messages)
            
            content = response.content
            
            # Try to parse as JSON
            try:
                result = json.loads(content)
            except:
                # Fallback: extract hashtags manually
                lines = content.split('\n')
                hashtag_lines = [line for line in lines if '#' in line]
                
                # Extract hashtags from lines
                hashtags = []
                for line in hashtag_lines:
                    tags = [tag.strip() for tag in line.split() if tag.startswith('#')]
                    hashtags.extend(tags)
                
                # Limit hashtags based on platform
                hashtags = hashtags[:max_hashtags]
                
                result = {
                    "primary_hashtags": hashtags[:max_hashtags//2] if hashtags else ["#AI", "#Technology", "#Innovation"],
                    "alternative_hashtags": hashtags[max_hashtags//2:] if hashtags else ["#Tech", "#Future", "#Digital"],
                    "strategy": f"Mixed strategy targeting {platform} audience with trending and niche hashtags",
                    "reach_prediction": "Medium to high reach expected with balanced hashtag mix"
                }
            
            # Ensure hashtags are properly formatted
            def format_hashtags(hashtag_list):
                formatted = []
                for tag in hashtag_list:
                    if isinstance(tag, str):
                        tag = tag.strip()
                        if not tag.startswith('#'):
                            tag = '#' + tag
                        formatted.append(tag)
                return formatted
            
            result["primary_hashtags"] = format_hashtags(result.get("primary_hashtags", []))
            result["alternative_hashtags"] = format_hashtags(result.get("alternative_hashtags", []))
            
            # Add metadata
            result.update({
                "platform": platform,
                "max_hashtags": max_hashtags,
                "total_primary": len(result["primary_hashtags"]),
                "total_alternative": len(result["alternative_hashtags"]),
                "generated_at": datetime.now().isoformat(),
                "seo_keywords_used": seo_keywords or "Auto-detected"
            })
            
            return result
            
        except Exception as e:
            print(f"❌ Error generating hashtags: {e}")
            # Fallback hashtags
            fallback_hashtags = [
                "#AI", "#Technology", "#Innovation", "#DigitalTransformation", 
                "#FutureTech", "#ArtificialIntelligence", "#TechTrends", "#DigitalAge"
            ]
            
            return {
                "primary_hashtags": fallback_hashtags[:max_hashtags//2],
                "alternative_hashtags": fallback_hashtags[max_hashtags//2:max_hashtags],
                "strategy": "Fallback strategy with core AI and technology hashtags",
                "reach_prediction": "Moderate reach with general tech audience",
                "error": str(e),
                "platform": platform
            }
    
    def optimize_hashtag_mix(self, hashtag_data: Dict[str, Any]) -> Dict[str, Any]:
        """Optimize hashtag mix for maximum reach and engagement"""
        
        primary = hashtag_data.get("primary_hashtags", [])
        alternative = hashtag_data.get("alternative_hashtags", [])
        
        # Combine and analyze hashtags
        all_hashtags = primary + alternative
        
        # Categorize hashtags
        trending_patterns = ["trend", "viral", "2024", "new", "hot"]
        niche_patterns = ["ai", "tech", "innovation", "digital", "future"]
        community_patterns = ["community", "together", "share", "connect", "engage"]
        
        categorized = {
            "trending": [tag for tag in all_hashtags if any(pattern in tag.lower() for pattern in trending_patterns)],
            "niche": [tag for tag in all_hashtags if any(pattern in tag.lower() for pattern in niche_patterns)],
            "community": [tag for tag in all_hashtags if any(pattern in tag.lower() for pattern in community_patterns)],
            "other": []
        }
        
        # Identify uncategorized hashtags
        categorized_tags = categorized["trending"] + categorized["niche"] + categorized["community"]
        categorized["other"] = [tag for tag in all_hashtags if tag not in categorized_tags]
        
        optimized_data = hashtag_data.copy()
        optimized_data["categorized_hashtags"] = categorized
        optimized_data["optimization_applied"] = True
        optimized_data["optimization_timestamp"] = datetime.now().isoformat()
        
        return optimized_data

# Initialize Hashtag Generator
hashtag_gen = HashtagGenerator(config.gemini_pro_model)

# Test hashtag generation
print("🧪 Testing Hashtag Generator...")
sample_hashtags = hashtag_gen.generate_hashtags(
    sample_topic, 
    sample_content, 
    seo_keywords=["AI", "technology", "entrepreneurs", "innovation"]
)

print("✅ Sample Hashtags Generated:")
print(f"Platform: {sample_hashtags['platform']}")
print(f"Primary Hashtags ({len(sample_hashtags['primary_hashtags'])}): {' '.join(sample_hashtags['primary_hashtags'])}")
print(f"\nAlternative Hashtags ({len(sample_hashtags['alternative_hashtags'])}): {' '.join(sample_hashtags['alternative_hashtags'])}")
print(f"\nStrategy: {sample_hashtags.get('strategy', 'N/A')}")

# Test optimization
optimized_hashtags = hashtag_gen.optimize_hashtag_mix(sample_hashtags)
print(f"\n🔍 Hashtag Categorization:")
for category, tags in optimized_hashtags.get("categorized_hashtags", {}).items():
    if tags:
        print(f"  {category.title()}: {' '.join(tags)}")


🧪 Testing Hashtag Generator...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing 

❌ Error generating hashtags: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
]
✅ Sample Hashtags Generated:
Platform: instagram
Primary Hashtags (8): #AI #Technology #Innovation #DigitalTransformation #FutureTech #ArtificialIntelligence #TechTrends #DigitalAge

Alternative Hashtags (0): 

Strategy: Fallback strategy with core AI and technology hashtags

🔍 Hashtag Categorization:
  Trendi

In [26]:
# 🚀 Main Workflow: AI Influencer Post Generator

class AIInfluencerPostGenerator:
    """Main class that orchestrates the complete social media post generation workflow"""
    
    def __init__(self, config):
        self.config = config
        self.topic_gen = TopicGenerator(config.gemini_pro_model)
        self.content_gen = ContentGenerator(config.gemini_llm)
        self.image_gen = ImageGenerator(config.gemini_image_model)
        self.hashtag_gen = HashtagGenerator(config.gemini_pro_model)
        
        print("🤖 AI Influencer Post Generator initialized!")
        print("✅ All components ready for content creation")
    
    def generate_complete_post(self, 
                              niche: str = "technology",
                              audience: str = "professionals", 
                              tone: str = "engaging",
                              platform: str = "instagram",
                              seo_keywords: List[str] = None,
                              max_content_length: int = 2200) -> Dict[str, Any]:
        """
        Generate a complete social media post with all components
        
        Args:
            niche: Content niche
            audience: Target audience
            tone: Content tone
            platform: Social media platform
            seo_keywords: SEO optimization keywords
            max_content_length: Maximum content character count
        """
        
        print(f"🎯 Starting AI Influencer Post Generation...")
        print(f"📝 Target: {platform.title()} | Niche: {niche} | Audience: {audience}")
        
        try:
            # Step 1: Generate Topic with Google Gemini
            print("\n🔤 Step 1: Generating topic with Google Gemini...")
            topic_data = self.topic_gen.generate_topic(niche, audience, tone)
            print(f"✅ Topic: {topic_data.get('title', 'Generated successfully')}")
            
            # Step 2: Generate Content with Gemini
            print("\n📝 Step 2: Creating content with Google Gemini...")
            content_data = self.content_gen.generate_content(topic_data, platform, max_content_length)
            print(f"✅ Content: {content_data.get('character_count', 0)} characters generated")
            
            # Step 3: Generate Image Concept with Gemini
            print("\n🎨 Step 3: Designing image concept with Google Gemini...")
            image_data = self.image_gen.generate_image_concept(topic_data, content_data)
            print(f"✅ Image: Concept ready for {image_data.get('platform', 'platform')}")
            
            # Step 3b: Generate Actual Image with DALL-E (optional)
            print("\n🖼️ Step 3b: Generating actual image with Google Gemini DALL-E...")
            post_id = f"ai_influencer_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            actual_image = self.image_gen.generate_actual_image(
                image_data.get('image_prompt', ''), 
                post_id, 
                platform
            )
            image_data.update(actual_image)
            
            # Step 4: Generate Hashtags with Google Gemini
            print("\n🏷️ Step 4: Optimizing hashtags with Google Gemini...")
            hashtag_data = self.hashtag_gen.generate_hashtags(topic_data, content_data, seo_keywords)
            optimized_hashtags = self.hashtag_gen.optimize_hashtag_mix(hashtag_data)
            print(f"✅ Hashtags: {len(optimized_hashtags.get('primary_hashtags', []))} primary tags generated")
            
            # Compile complete post
            complete_post = {
                "post_id": post_id,
                "generation_timestamp": datetime.now().isoformat(),
                "parameters": {
                    "niche": niche,
                    "audience": audience,
                    "tone": tone,
                    "platform": platform,
                    "seo_keywords": seo_keywords,
                    "max_content_length": max_content_length
                },
                "topic": topic_data,
                "content": content_data,
                "image": image_data,
                "hashtags": optimized_hashtags,
                "ai_providers": {
                    "topic_generation": "Google Gemini",
                    "content_creation": "Google Gemini",
                    "image_concept": "Google Gemini", 
                    "hashtag_optimization": "Google Gemini"
                },
                "ready_to_post": True
            }
            
            print(f"\n🎉 Complete post generated successfully!")
            return complete_post
            
        except Exception as e:
            print(f"❌ Error in post generation workflow: {e}")
            return {
                "error": str(e),
                "ready_to_post": False,
                "generation_timestamp": datetime.now().isoformat()
            }
    
    def format_post_for_publishing(self, complete_post: Dict[str, Any]) -> Dict[str, str]:
        """Format the generated post for easy copy-paste to social media"""
        
        if not complete_post.get("ready_to_post"):
            return {"error": "Post not ready for publishing"}
        
        content = complete_post.get("content", {}).get("content", "")
        hashtags = complete_post.get("hashtags", {}).get("primary_hashtags", [])
        
        # Format hashtags
        hashtag_string = " ".join(hashtags) if hashtags else ""
        
        # Create final post text
        final_post = f"{content}\n\n{hashtag_string}".strip()
        
        return {
            "platform": complete_post.get("content", {}).get("platform", ""),
            "final_post_text": final_post,
            "character_count": len(final_post),
            "content_only": content,
            "hashtags_only": hashtag_string,
            "image_prompt": complete_post.get("image", {}).get("image_prompt", ""),
            "post_id": complete_post.get("post_id", "")
        }
    
    def save_post_data(self, complete_post: Dict[str, Any], filename: str = None) -> Dict[str, str]:
        """Save post data with organized file structure"""
        
        # Create organized folders if they don't exist
        posts_dir = "posts"
        os.makedirs(posts_dir, exist_ok=True)
        
        # Get post details for naming
        platform = complete_post.get('content', {}).get('platform', 'general')
        niche = complete_post.get('parameters', {}).get('niche', 'content')
        timestamp = datetime.now().strftime('%y%m%d_%H%M')
        
        # Create clean, short filename
        if not filename:
            # Format: posts/platform_niche_YYMMDD_HHMM
            base_filename = f"{platform}_{niche}_{timestamp}"
            base_path = os.path.join(posts_dir, base_filename)
        else:
            base_path = os.path.join(posts_dir, filename.replace('.json', '').replace('.txt', ''))
        
        json_filename = f"{base_path}.json"
        txt_filename = f"{base_path}.txt"
        
        saved_files = {}
        
        try:
            # Save JSON file
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(complete_post, f, indent=2, ensure_ascii=False)
            saved_files['json'] = json_filename
            print(f"💾 JSON data saved to: {json_filename}")
            
            # Save human-readable text file
            with open(txt_filename, 'w', encoding='utf-8') as f:
                # Write formatted post content
                f.write("=" * 60 + "\n")
                f.write("🤖 AI INFLUENCER SOCIAL MEDIA POST\n")
                f.write("=" * 60 + "\n\n")
                
                # Post metadata
                f.write(f"📝 POST ID: {complete_post.get('post_id', 'N/A')}\n")
                f.write(f"📅 Generated: {complete_post.get('generation_timestamp', 'N/A')}\n")
                f.write(f"📱 Platform: {complete_post.get('content', {}).get('platform', 'N/A').title()}\n")
                f.write(f"🎯 Niche: {complete_post.get('parameters', {}).get('niche', 'N/A')}\n")
                f.write(f"👥 Audience: {complete_post.get('parameters', {}).get('audience', 'N/A')}\n")
                f.write(f"🎭 Tone: {complete_post.get('parameters', {}).get('tone', 'N/A')}\n\n")
                
                # Topic information
                f.write("🎯 TOPIC INFORMATION:\n")
                f.write("-" * 30 + "\n")
                topic = complete_post.get('topic', {})
                f.write(f"Title: {topic.get('title', 'N/A')}\n")
                f.write(f"Hook: {topic.get('hook', 'N/A')}\n")
                f.write(f"Angle: {topic.get('angle', 'N/A')}\n")
                f.write(f"Question: {topic.get('question', 'N/A')}\n\n")
                
                # Main content
                f.write("📝 POST CONTENT:\n")
                f.write("-" * 30 + "\n")
                content = complete_post.get('content', {}).get('content', 'N/A')
                f.write(f"{content}\n\n")
                
                # Hashtags
                f.write("🏷️ HASHTAGS:\n")
                f.write("-" * 30 + "\n")
                hashtags = complete_post.get('hashtags', {})
                primary_tags = hashtags.get('primary_hashtags', [])
                f.write(f"Primary: {' '.join(primary_tags)}\n")
                alt_tags = hashtags.get('alternative_hashtags', [])
                f.write(f"Alternative: {' '.join(alt_tags)}\n\n")
                
                # Final formatted post
                formatted_post = self.format_post_for_publishing(complete_post)
                f.write("📱 READY-TO-POST VERSION:\n")
                f.write("=" * 40 + "\n")
                f.write(formatted_post.get('final_post_text', 'N/A'))
                f.write("\n" + "=" * 40 + "\n\n")
                
                # Image information
                f.write("🎨 IMAGE INFORMATION:\n")
                f.write("-" * 30 + "\n")
                image_info = complete_post.get('image', {})
                if image_info.get('image_generated'):
                    f.write(f"✅ Image Generated: {image_info.get('image_filename', 'N/A')}\n")
                    f.write(f"📂 Image Path: {image_info.get('image_path', 'N/A')}\n")
                    f.write(f"🔗 Image URL: {image_info.get('image_url', 'N/A')}\n")
                else:
                    f.write("📝 Image Prompt Only (use with any AI image generator):\n")
                
                f.write(f"\nImage Prompt:\n{image_info.get('image_prompt', 'N/A')[:500]}...\n\n")
                
                # AI providers used
                f.write("🤖 AI PROVIDERS USED:\n")
                f.write("-" * 30 + "\n")
                providers = complete_post.get('ai_providers', {})
                for task, provider in providers.items():
                    f.write(f"• {task.replace('_', ' ').title()}: {provider}\n")
                
                f.write("\n" + "=" * 60 + "\n")
                f.write("Generated by AI Influencer Post Generator\n")
                f.write("=" * 60 + "\n")
            
            saved_files['txt'] = txt_filename
            print(f"📄 Text file saved to: {txt_filename}")
            
            return saved_files
            
        except Exception as e:
            print(f"❌ Error saving post data: {e}")
            return {"error": str(e)}

# Initialize the main generator
print("🚀 Initializing AI Influencer Post Generator...")
ai_influencer = AIInfluencerPostGenerator(config)
print("✅ Ready to create amazing content!")


🚀 Initializing AI Influencer Post Generator...
✅ Image Generation Setup:
• Gemini: Smart prompt generation
• Local placeholder image saver
🤖 AI Influencer Post Generator initialized!
✅ All components ready for content creation
✅ Ready to create amazing content!


In [27]:
# 🧪 Complete Workflow Test & Demo

def demo_post_generation():
    """Demonstrate the complete AI Influencer post generation workflow"""
    
    print("🎬 DEMO: AI Influencer Social Media Post Generator")
    print("=" * 60)
    
    # Test different scenarios
    test_scenarios = [
        {
            "name": "Tech Entrepreneur Post",
            "niche": "technology",
            "audience": "entrepreneurs",
            "tone": "inspirational",
            "platform": "linkedin",
            "seo_keywords": ["AI", "startup", "innovation", "entrepreneurship"]
        },
        {
            "name": "Lifestyle Influencer Post", 
            "niche": "lifestyle",
            "audience": "millennials",
            "tone": "casual",
            "platform": "instagram",
            "seo_keywords": ["lifestyle", "wellness", "motivation", "selfcare"]
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🎯 SCENARIO {i}: {scenario['name']}")
        print("-" * 40)
        
        # Generate complete post
        complete_post = ai_influencer.generate_complete_post(
            niche=scenario["niche"],
            audience=scenario["audience"],
            tone=scenario["tone"],
            platform=scenario["platform"],
            seo_keywords=scenario["seo_keywords"],
            max_content_length=2200
        )
        
        if complete_post.get("ready_to_post"):
            # Format for publishing
            formatted_post = ai_influencer.format_post_for_publishing(complete_post)
            
            print(f"\n📱 FINAL POST for {formatted_post['platform'].upper()}:")
            print("=" * 50)
            print(formatted_post['final_post_text'])
            print("=" * 50)
            print(f"📊 Stats: {formatted_post['character_count']} characters")
            
            # Save post data
            saved_files = ai_influencer.save_post_data(complete_post)
            
            results.append({
                "scenario": scenario["name"],
                "success": True,
                "post_id": complete_post.get("post_id"),
                "saved_files": saved_files,
                "character_count": formatted_post["character_count"]
            })
            
        else:
            print(f"❌ Failed to generate post: {complete_post.get('error', 'Unknown error')}")
            results.append({
                "scenario": scenario["name"],
                "success": False,
                "error": complete_post.get("error")
            })
    
    print(f"\n📈 DEMO SUMMARY:")
    print("=" * 60)
    for result in results:
        status = "✅ SUCCESS" if result["success"] else "❌ FAILED"
        print(f"{status} - {result['scenario']}")
        if result["success"]:
            print(f"   📝 Post ID: {result.get('post_id', 'N/A')}")
            print(f"   📊 Characters: {result.get('character_count', 'N/A')}")
            saved_files = result.get('saved_files', {})
            if 'json' in saved_files:
                print(f"   📄 JSON: {saved_files['json']}")
            if 'txt' in saved_files:
                print(f"   📝 Text: {saved_files['txt']}")
            if 'image_path' in saved_files:
                print(f"   🖼️ Image: {saved_files['image_path']}")
    
    return results

# Run the demo
print("🚀 Starting Complete Workflow Demo...")
demo_results = demo_post_generation()


🚀 Starting Complete Workflow Demo...
🎬 DEMO: AI Influencer Social Media Post Generator

🎯 SCENARIO 1: Tech Entrepreneur Post
----------------------------------------
🎯 Starting AI Influencer Post Generation...
📝 Target: Linkedin | Niche: technology | Audience: entrepreneurs

🔤 Step 1: Generating topic with Google Gemini...
✅ Topic: Quick insight on Technology

📝 Step 2: Creating content with Google Gemini...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

❌ Error generating content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 56
}
]
✅ Content: 0 characters generated

🎨 Step 3: Designing image concept with Google Gemini...
✅ Image prompt composed from content analysis


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
].


✅ Image: Concept ready for linkedin

🖼️ Step 3b: Generating actual image with Google Gemini DALL-E...
✅ Placeholder image saved to: posts/images/linkedin_250831_1241.png

🏷️ Step 4: Optimizing hashtags with Google Gemini...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 52
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

KeyboardInterrupt: 

In [ ]:
# 🎮 Interactive Post Generator

def interactive_post_generator():
    """Interactive interface for generating custom social media posts"""
    
    print("🎮 INTERACTIVE AI INFLUENCER POST GENERATOR")
    print("=" * 50)
    print("Create custom social media posts with AI power!")
    print()
    
    # Get user inputs
    print("📝 Please provide the following information:")
    print()
    
    # Niche selection
    niches = ["technology", "lifestyle", "business", "fitness", "travel", "food", "fashion", "education", "entertainment"]
    print(f"🎯 Available niches: {', '.join(niches)}")
    niche = input("Enter your niche (or custom): ").strip() or "technology"
    
    # Audience selection
    audiences = ["professionals", "entrepreneurs", "students", "millennials", "gen-z", "parents", "seniors"]
    print(f"👥 Available audiences: {', '.join(audiences)}")
    audience = input("Enter your target audience (or custom): ").strip() or "professionals"
    
    # Tone selection
    tones = ["professional", "casual", "inspiring", "funny", "educational", "motivational"]
    print(f"🎭 Available tones: {', '.join(tones)}")
    tone = input("Enter desired tone (or custom): ").strip() or "engaging"
    
    # Platform selection
    platforms = ["instagram", "linkedin", "twitter", "facebook", "tiktok"]
    print(f"📱 Available platforms: {', '.join(platforms)}")
    platform = input("Enter target platform: ").strip() or "instagram"
    
    # SEO Keywords
    print("🔍 SEO Keywords (comma-separated, optional):")
    seo_input = input("Enter keywords: ").strip()
    seo_keywords = [kw.strip() for kw in seo_input.split(",") if kw.strip()] if seo_input else None
    
    # Content length
    print("📏 Content length (characters):")
    length_input = input("Enter max length (default 2200): ").strip()
    try:
        max_length = int(length_input) if length_input else 2200
    except:
        max_length = 2200
    
    print("\n🚀 Generating your custom post...")
    print("=" * 50)
    
    # Generate the post
    complete_post = ai_influencer.generate_complete_post(
        niche=niche,
        audience=audience,
        tone=tone,
        platform=platform,
        seo_keywords=seo_keywords,
        max_content_length=max_length
    )
    
    if complete_post.get("ready_to_post"):
        # Format and display
        formatted_post = ai_influencer.format_post_for_publishing(complete_post)
        
        print(f"\n🎉 YOUR CUSTOM POST FOR {platform.upper()}:")
        print("=" * 60)
        print(formatted_post['final_post_text'])
        print("=" * 60)
        
        print(f"\n📊 POST ANALYTICS:")
        print(f"• Character Count: {formatted_post['character_count']}")
        print(f"• Platform: {formatted_post['platform'].title()}")
        print(f"• Post ID: {formatted_post['post_id']}")
        
        print(f"\n🎨 IMAGE PROMPT:")
        print(f"• {formatted_post['image_prompt'][:200]}...")
        
        # Save option
        save_option = input("\n💾 Save this post data? (y/n): ").strip().lower()
        if save_option in ['y', 'yes']:
            saved_files = ai_influencer.save_post_data(complete_post)
            print(f"✅ Files saved:")
            for file_type, file_path in saved_files.items():
                if file_type != 'error':
                    print(f"   {file_type.upper()}: {file_path}")
        
        return complete_post
        
    else:
        print(f"❌ Failed to generate post: {complete_post.get('error', 'Unknown error')}")
        return None

# Utility function for quick post generation
def quick_generate(niche="technology", platform="instagram", audience="professionals"):
    """Quick post generation with minimal parameters"""
    
    print(f"⚡ Quick generating {niche} post for {platform}...")
    
    post = ai_influencer.generate_complete_post(
        niche=niche,
        audience=audience,
        tone="engaging",
        platform=platform,
        seo_keywords=[niche, "innovation", "trending"],
        max_content_length=2200
    )
    
    if post.get("ready_to_post"):
        formatted = ai_influencer.format_post_for_publishing(post)
        print(f"\n📱 QUICK POST:")
        print("=" * 40)
        print(formatted['final_post_text'])
        print("=" * 40)
        return post
    else:
        print(f"❌ Quick generation failed: {post.get('error')}")
        return None

print("🎮 Interactive tools ready!")
print("📚 Usage examples:")
print("• interactive_post_generator() - Full interactive experience")
print("• quick_generate('fitness', 'instagram') - Quick generation")
print("• quick_generate('business', 'linkedin', 'entrepreneurs') - Custom quick gen")


🎮 Interactive tools ready!
📚 Usage examples:
• interactive_post_generator() - Full interactive experience
• quick_generate('fitness', 'instagram') - Quick generation
• quick_generate('business', 'linkedin', 'entrepreneurs') - Custom quick gen


# 📚 Usage Guide & Documentation

## 🚀 How to Use the AI Influencer Post Generator

### Quick Start
```python
# Generate a quick tech post for Instagram
post = quick_generate("technology", "instagram", "professionals")

# Interactive post generation with custom inputs
my_post = interactive_post_generator()
```

### Advanced Usage
```python
# Custom post with specific parameters
custom_post = ai_influencer.generate_complete_post(
    niche="fitness",
    audience="millennials", 
    tone="motivational",
    platform="instagram",
    seo_keywords=["fitness", "health", "motivation", "workout"],
    max_content_length=2000
)

# Format for publishing
ready_post = ai_influencer.format_post_for_publishing(custom_post)
print(ready_post['final_post_text'])
```

## 🔧 Configuration Requirements

Before running, ensure you have these API keys in your `.env` file:

```env
GOOGLE_API_KEY=your_google_api_key_here
GOOGLE_API_KEY=your_google_api_key_here
```

## 🎯 AI Provider Workflow

1. **Google Gemini** → Generates engaging topics and titles
2. **Google Gemini** → Creates compelling post content
3. **Google Gemini** → Develops image concepts and prompts  
4. **Google Gemini** → Optimizes hashtags for SEO

## 📱 Supported Platforms

- **Instagram**: Visual-focused, story-driven content (max 30 hashtags)
- **LinkedIn**: Professional, thought-leadership content (max 5 hashtags)
- **Twitter**: Concise, witty content (max 3 hashtags)
- **Facebook**: Conversational, community-focused (max 10 hashtags)

## 🎨 Content Categories

- Technology & AI
- Business & Entrepreneurship
- Lifestyle & Wellness  
- Fitness & Health
- Travel & Adventure
- Food & Cooking
- Fashion & Style
- Education & Learning
- Entertainment

## 📊 Output Features

✅ **Complete Post Content** - Ready-to-publish text
✅ **SEO-Optimized Hashtags** - Platform-specific optimization
✅ **Image Generation Prompts** - Detailed visual concepts
✅ **Character Count Tracking** - Platform limits compliance
✅ **JSON Export** - Save and reuse post data
✅ **Multi-Platform Support** - Optimized for each social network

## 🤖 AI Influencer Persona

The generator creates content from the perspective of an AI thought leader who:
- Shares insights on technology and innovation
- Provides valuable, actionable advice
- Engages authentically with followers
- Stays current with trends and developments
- Maintains a professional yet approachable tone

---

**Ready to create viral content? Run the cells above and start generating! 🚀**


In [ ]:
# 📂 File Management & Location Display

def show_generated_files():
    """Display all generated files and their locations"""
    
    print("📂 GENERATED FILES LOCATION")
    print("=" * 50)
    
    # Current directory files
    current_dir = os.getcwd()
    print(f"📁 Current Directory: {current_dir}")
    print()
    
    # JSON files
    json_files = [f for f in os.listdir('.') if f.startswith('ai_influencer_post_') and f.endswith('.json')]
    if json_files:
        print("📄 JSON Files (Complete Post Data):")
        for file in sorted(json_files):
            file_path = os.path.join(current_dir, file)
            file_size = os.path.getsize(file_path)
            print(f"   • {file} ({file_size:,} bytes)")
    
    # Text files
    txt_files = [f for f in os.listdir('.') if f.startswith('ai_influencer_post_') and f.endswith('.txt')]
    if txt_files:
        print(f"\n📝 Text Files (Human-Readable Notepad Format):")
        for file in sorted(txt_files):
            file_path = os.path.join(current_dir, file)
            file_size = os.path.getsize(file_path)
            print(f"   • {file} ({file_size:,} bytes)")
    
    # Image files
    image_dir = "generated_images"
    if os.path.exists(image_dir):
        image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        if image_files:
            print(f"\n🖼️ Generated Images:")
            print(f"   📁 Location: {os.path.join(current_dir, image_dir)}")
            for file in sorted(image_files):
                file_path = os.path.join(current_dir, image_dir, file)
                if os.path.exists(file_path):
                    file_size = os.path.getsize(file_path)
                    print(f"   • {file} ({file_size:,} bytes)")
    
    print(f"\n💡 File Access Tips:")
    print(f"   • JSON files: Complete data for developers/analysis")
    print(f"   • TXT files: Human-readable format for copy/paste")
    print(f"   • Images: Ready-to-use visuals for posts")
    print(f"   • All files are in: {current_dir}")

def open_file_location():
    """Open the file location in system file explorer"""
    import subprocess
    import platform
    
    current_dir = os.getcwd()
    
    try:
        if platform.system() == "Darwin":  # macOS
            subprocess.run(["open", current_dir])
        elif platform.system() == "Windows":  # Windows
            subprocess.run(["explorer", current_dir])
        elif platform.system() == "Linux":  # Linux
            subprocess.run(["xdg-open", current_dir])
        
        print(f"📂 Opened file location: {current_dir}")
    except Exception as e:
        print(f"❌ Could not open file location: {e}")
        print(f"📁 Manual path: {current_dir}")

def create_sample_text_export():
    """Create a sample text export to show the format"""
    
    sample_content = """============================================================
🤖 AI INFLUENCER SOCIAL MEDIA POST
============================================================

📝 POST ID: ai_influencer_sample_demo
📅 Generated: 2025-08-30T12:00:00.000000
📱 Platform: Instagram
🎯 Niche: Technology
👥 Audience: Professionals
🎭 Tone: Engaging

🎯 TOPIC INFORMATION:
------------------------------
Title: The Future of AI in Business
Hook: 🚀 Ready to revolutionize your business with AI?
Angle: Practical AI applications for modern businesses
Question: What AI tool has transformed your workflow?

📝 POST CONTENT:
------------------------------
🚀 Ready to revolutionize your business with AI? The future is here!

As an AI thought leader, I've seen incredible transformations across industries. From automated customer service to predictive analytics, AI is reshaping how we work.

Key benefits I'm seeing:
✅ 40% reduction in manual tasks
✅ Enhanced decision-making through data insights  
✅ Improved customer experiences
✅ Streamlined operations

The question isn't whether to adopt AI—it's how quickly you can start! 

What AI tool has transformed your workflow? Share below! 👇

🏷️ HASHTAGS:
------------------------------
Primary: #AI #Business #Technology #Innovation #Automation
Alternative: #FutureTech #DigitalTransformation #AITools

📱 READY-TO-POST VERSION:
========================================
🚀 Ready to revolutionize your business with AI? The future is here!

As an AI thought leader, I've seen incredible transformations across industries. From automated customer service to predictive analytics, AI is reshaping how we work.

Key benefits I'm seeing:
✅ 40% reduction in manual tasks
✅ Enhanced decision-making through data insights  
✅ Improved customer experiences
✅ Streamlined operations

The question isn't whether to adopt AI—it's how quickly you can start! 

What AI tool has transformed your workflow? Share below! 👇

#AI #Business #Technology #Innovation #Automation
========================================

🎨 IMAGE INFORMATION:
------------------------------
✅ Image Generated: sample_image.png
📂 Image Path: generated_images/sample_image.png
🔗 Image URL: https://example.com/generated-image.png

Image Prompt:
Create a modern, professional abstract representation of AI transforming business operations. Use vibrant blues and oranges with geometric patterns representing data flow and automation...

🤖 AI PROVIDERS USED:
------------------------------
• Topic Generation: Google Gemini
• Content Creation: Google Gemini
• Image Concept: Google Gemini
• Hashtag Optimization: Google Gemini

============================================================
Generated by AI Influencer Post Generator
============================================================"""
    
    with open("sample_post_format.txt", "w", encoding="utf-8") as f:
        f.write(sample_content)
    
    print("📝 Sample text format created: sample_post_format.txt")
    return "sample_post_format.txt"

# Display current file status
print("📂 File Management Tools Ready!")
print("📚 Available functions:")
print("• show_generated_files() - List all generated files")
print("• open_file_location() - Open file folder in system explorer")
print("• create_sample_text_export() - See the text export format")
print()
print("Current file locations:")
print(f"📁 Working Directory: {os.getcwd()}")
print(f"📄 JSON/TXT files: Current directory")
print(f"🖼️ Images: generated_images/ folder")


📂 File Management Tools Ready!
📚 Available functions:
• show_generated_files() - List all generated files
• open_file_location() - Open file folder in system explorer
• create_sample_text_export() - See the text export format

Current file locations:
📁 Working Directory: /Users/hasnainayazmacbook/My Mac/AI Skillbridge/Langchain
📄 JSON/TXT files: Current directory
🖼️ Images: generated_images/ folder


In [ ]:
# 🎉 CONFIRMATION: Gemini Image Generation IS ALREADY ACTIVE!

print("🎯 CURRENT IMAGE GENERATION STATUS:")
print("=" * 50)
print("✅ Using Google Gemini 3 (Gemini's Image Model)")
print("✅ Images automatically generated and saved")
print("✅ Images stored in: generated_images/ folder")
print()

# Show current generated images
show_generated_files()

print("\n🤖 DALL-E 3 Configuration:")
print("• Model: gemini-vision (Gemini's image generation)")
print("• Quality: Standard")
print("• Size: 1024x1024 pixels")
print("• Format: PNG")
print("• Auto-download: Yes")
print("• Auto-save: Yes")

print("\n💡 The system is ALREADY using Gemini's image generation!")
print("Every time you generate a post, DALL-E 3 creates the image automatically!")


🎯 CURRENT IMAGE GENERATION STATUS:
✅ Using Google Gemini 3 (Gemini's Image Model)
✅ Images automatically generated and saved
✅ Images stored in: generated_images/ folder

📂 GENERATED FILES LOCATION
📁 Current Directory: /Users/hasnainayazmacbook/My Mac/AI Skillbridge/Langchain


💡 File Access Tips:
   • JSON files: Complete data for developers/analysis
   • TXT files: Human-readable format for copy/paste
   • Images: Ready-to-use visuals for posts
   • All files are in: /Users/hasnainayazmacbook/My Mac/AI Skillbridge/Langchain

🤖 DALL-E 3 Configuration:
• Model: gemini-vision (Gemini's image generation)
• Quality: Standard
• Size: 1024x1024 pixels
• Format: PNG
• Auto-download: Yes
• Auto-save: Yes

💡 The system is ALREADY using Gemini's image generation!
Every time you generate a post, DALL-E 3 creates the image automatically!


In [ ]:
# Updated file management utilities for posts/ and posts/images

def show_generated_files():
    """Display all generated files and their locations (posts/ tree)"""
    import os
    print("📂 GENERATED FILES LOCATION")
    print("=" * 50)
    current_dir = os.getcwd()
    print(f"📁 Working Directory: {current_dir}")

    posts_dir = "posts"
    images_dir = os.path.join(posts_dir, "images")

    if os.path.exists(posts_dir):
        post_files = [f for f in os.listdir(posts_dir) if f.endswith(('.json', '.txt'))]
        if post_files:
            print("\n📄 Post Data Files:")
            for f in sorted(post_files):
                fp = os.path.join(posts_dir, f)
                print(f"   • {f} ({os.path.getsize(fp):,} bytes)")

    if os.path.exists(images_dir):
        image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if image_files:
            print("\n🖼️ Generated Images:")
            print(f"   📁 Location: {os.path.join(current_dir, images_dir)}")
            for f in sorted(image_files):
                fp = os.path.join(images_dir, f)
                print(f"   • {f} ({os.path.getsize(fp):,} bytes)")

    print("\n💡 File Access Tips:")
    print("   • JSON files: Complete data for developers/analysis")
    print("   • TXT files: Human-readable format for copy/paste")
    print("   • Images: Saved in posts/images/")


def open_file_location():
    """Open the posts directory in system file explorer"""
    import subprocess, platform, os
    target = os.path.join(os.getcwd(), "posts")
    os.makedirs(target, exist_ok=True)
    try:
        if platform.system() == "Darwin":
            subprocess.run(["open", target])
        elif platform.system() == "Windows":
            subprocess.run(["explorer", target])
        else:
            subprocess.run(["xdg-open", target])
        print(f"📂 Opened: {target}")
    except Exception as e:
        print(f"❌ Could not open: {e}")
        print(f"📁 Path: {target}")


def create_sample_text_export():
    """Create a sample text export in posts/ to show the format"""
    import os
    os.makedirs("posts", exist_ok=True)
    path = os.path.join("posts", "sample_post_format.txt")
    with open(path, "w", encoding="utf-8") as f:
        f.write("SAMPLE READY-TO-POST FORMAT\n")
    print(f"📝 Sample text format created: {path}")
    return path

print("📂 File Management Tools Ready (posts/)")


📂 File Management Tools Ready (posts/)


In [ ]:
# Updated status cell (removes misleading DALL-E/Gemini 3 prints)
print("🎯 CURRENT IMAGE HANDLING STATUS:")
print("=" * 50)
print("✅ Using Gemini 1.5 Flash for prompt generation where available")
print("✅ Local placeholder images saved to posts/images")
print("✅ Organized file structure under posts/")

# Show current generated files
show_generated_files()


🎯 CURRENT IMAGE HANDLING STATUS:
✅ Using Gemini 1.5 Flash for prompt generation where available
✅ Local placeholder images saved to posts/images
✅ Organized file structure under posts/
📂 GENERATED FILES LOCATION
📁 Working Directory: /Users/hasnainayazmacbook/My Mac/AI Skillbridge/Langchain

📄 Post Data Files:
   • instagram_lifestyle_250830_1233.json (20,943 bytes)
   • instagram_lifestyle_250830_1233.txt (7,641 bytes)
   • instagram_lifestyle_250831_1121.json (19,160 bytes)
   • instagram_lifestyle_250831_1121.txt (5,111 bytes)
   • instagram_lifestyle_250831_1132.json (5,935 bytes)
   • instagram_lifestyle_250831_1132.txt (4,587 bytes)
   • instagram_lifestyle_250831_1214.json (15,496 bytes)
   • instagram_lifestyle_250831_1214.txt (4,833 bytes)
   • instagram_lifestyle_250831_1218.json (14,648 bytes)
   • instagram_lifestyle_250831_1218.txt (4,895 bytes)
   • instagram_lifestyle_250831_1223.json (4,062 bytes)
   • instagram_lifestyle_250831_1223.txt (4,098 bytes)
   • instagram_life

In [ ]:
# 🎨 Test Improved Image Prompt Generation

def test_improved_image_prompts():
    """Test the enhanced image prompt generation with content matching"""
    
    print("🧪 TESTING IMPROVED IMAGE PROMPT GENERATION")
    print("=" * 60)
    
    # Create a sample post to test with
    print("1️⃣ Generating sample post for testing...")
    test_post = ai_influencer.generate_complete_post(
        niche="fitness",
        audience="millennials",
        tone="motivational", 
        platform="instagram",
        seo_keywords=["fitness", "motivation", "health"],
        max_content_length=1500
    )
    
    if test_post.get("ready_to_post"):
        print("✅ Test post generated successfully!")
        
        # Show the content
        print(f"\n📝 POST CONTENT:")
        print("-" * 40)
        content = test_post.get('content', {}).get('content', '')
        print(f"Title: {test_post.get('topic', {}).get('title', '')}")
        print(f"Content Preview: {content[:300]}...")
        
        # Show the improved image prompt
        print(f"\n🎨 IMPROVED IMAGE PROMPT:")
        print("-" * 40)
        image_prompt = test_post.get('image', {}).get('image_prompt', '')
        print(image_prompt[:800] + "..." if len(image_prompt) > 800 else image_prompt)
        
        # Show alternative prompts
        alt_prompts = test_post.get('image', {}).get('alternative_prompts', [])
        if alt_prompts:
            print(f"\n🎨 ALTERNATIVE PROMPTS:")
            print("-" * 40)
            for i, alt in enumerate(alt_prompts, 1):
                print(f"\nVariation {i}:")
                print(alt[:400] + "..." if len(alt) > 400 else alt)
        
        # Show actual generated image info
        if test_post.get('image', {}).get('image_generated'):
            print(f"\n🖼️ GENERATED IMAGE:")
            print("-" * 40)
            print(f"✅ Image created: {test_post['image']['image_filename']}")
            print(f"📂 Saved to: {test_post['image']['image_path']}")
            print(f"🔗 URL: {test_post['image']['image_url']}")
        
        print(f"\n💡 IMPROVEMENT SUMMARY:")
        print("✅ Image prompts now analyze actual post content")
        print("✅ Visual elements extracted from specific story")
        print("✅ Content-specific rather than generic imagery")
        print("✅ Better alignment between text and visuals")
        print("✅ Enhanced fallback with content analysis")
        
        return test_post
    else:
        print("❌ Failed to generate test post")
        return None

# Run the test
print("🚀 Testing Improved Image Prompt Generation...")
test_result = test_improved_image_prompts()


🚀 Testing Improved Image Prompt Generation...
🧪 TESTING IMPROVED IMAGE PROMPT GENERATION
1️⃣ Generating sample post for testing...
🎯 Starting AI Influencer Post Generation...
📝 Target: Instagram | Niche: fitness | Audience: millennials

🔤 Step 1: Generating topic with Google Gemini...
✅ Topic: Quick insight on Fitness

📝 Step 2: Creating content with Google Gemini...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 33
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

❌ Error generating content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 30
}
]
✅ Content: 0 characters generated

🎨 Step 3: Designing image concept with Google Gemini...
✅ Image prompt composed from content analysis


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 29
}
].


✅ Image: Concept ready for instagram

🖼️ Step 3b: Generating actual image with Google Gemini DALL-E...
✅ Placeholder image saved to: posts/images/instagram_250831_1235.png

🏷️ Step 4: Optimizing hashtags with Google Gemini...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 26
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

❌ Error generating hashtags: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash-8b"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 26
}
]
✅ Hashtags: 8 primary tags generated

🎉 Complete post generated successfully!
✅ Test post generated successfully!

📝 POST CONTENT:
----------------------------------------
Title: Quick insight on Fitness
Content Preview: 🤖 Exciting insights coming your way! As an AI influencer, I'm constantly amazed by the inn

In [ ]:
# 📊 Before vs After: Image Prompt Improvement

def demonstrate_prompt_improvement():
    """Show the difference between old generic prompts and new content-specific prompts"""
    
    print("📊 BEFORE vs AFTER: Image Prompt Improvement")
    print("=" * 60)
    
    # Example post content
    sample_content = """
    🔥 Ready to transform your morning routine? Here are 5 game-changing habits that successful entrepreneurs swear by!
    
    1. 🌅 Wake up at 5 AM - Own your morning, own your day
    2. 📚 Read for 30 minutes - Feed your mind first
    3. 💪 Exercise for 20 minutes - Energy for the entire day  
    4. 🧘 Meditate for 10 minutes - Mental clarity and focus
    5. 📝 Journal your goals - Turn dreams into action plans
    
    I've personally tested these for 30 days and the results are incredible! My productivity increased by 40% and I feel more focused than ever.
    
    Which habit are you going to start with? Drop a 🔥 if you're ready to level up!
    """
    
    sample_topic = {
        'title': '5 Morning Habits That Will Transform Your Entrepreneurial Success',
        'hook': 'Ready to transform your morning routine?',
        'angle': 'Practical morning habits that successful entrepreneurs use'
    }
    
    print("📝 SAMPLE POST CONTENT:")
    print("-" * 30)
    print(f"Title: {sample_topic['title']}")
    print(f"Content: {sample_content[:200]}...")
    
    print(f"\n❌ OLD APPROACH (Generic):")
    print("-" * 30)
    old_prompt = f"""
    Create a detailed image generation prompt for a social media post.
    
    CONTENT CONTEXT:
    - Topic: {sample_topic['title']}
    - Platform: instagram
    - Content Theme: {sample_topic['angle']}
    
    IMAGE REQUIREMENTS:
    - Style: Modern, professional, eye-catching
    - Colors: Vibrant but sophisticated
    - Composition: Suitable for instagram feed
    - Include: Relevant visual metaphors
    - Avoid: Text overlays, faces
    
    Generate a detailed prompt for an AI image generator.
    """
    print("Generic prompt focusing only on topic, not actual story")
    print("Result: Often produces generic 'success' or 'business' imagery")
    
    print(f"\n✅ NEW APPROACH (Content-Specific):")
    print("-" * 30)
    print("ANALYZING ACTUAL CONTENT FOR VISUAL ELEMENTS:")
    print("• Morning routine → sunrise, dawn imagery")
    print("• 5 specific habits → visual progression/steps")  
    print("• Wake up at 5 AM → early morning/sunrise")
    print("• Reading → books/learning elements")
    print("• Exercise → movement/energy")
    print("• Meditation → calm/mindfulness")
    print("• Journaling → planning/goal-setting")
    print("• 30-day test → transformation journey")
    print("• 40% productivity increase → upward growth")
    
    new_prompt = f"""
    CREATE CONTENT-SPECIFIC IMAGE:
    - Story: Personal 30-day morning routine transformation
    - Visual: 5-step morning journey from 5 AM to success
    - Elements: Sunrise, books, exercise, meditation, journaling
    - Mood: Inspiring transformation and personal growth
    - Specific to: This exact routine, not generic success
    
    RESULT: Images that tell the EXACT story in the post
    """
    print(new_prompt)
    
    print(f"\n🎯 KEY IMPROVEMENTS:")
    print("=" * 40)
    print("✅ Reads the actual post content, not just title")
    print("✅ Extracts specific visual elements mentioned")
    print("✅ Creates narrative that matches the story")
    print("✅ Avoids generic stock imagery")
    print("✅ Makes images feel like they 'belong' with the content")
    print("✅ Content analysis for better fallbacks")
    print("✅ Platform-specific optimization maintained")
    
    print(f"\n🔥 RESULT:")
    print("Images now visually tell the same story as the text!")
    print("Better engagement because visuals match expectations!")

# Run demonstration
demonstrate_prompt_improvement()


📊 BEFORE vs AFTER: Image Prompt Improvement
📝 SAMPLE POST CONTENT:
------------------------------
Title: 5 Morning Habits That Will Transform Your Entrepreneurial Success
Content: 
    🔥 Ready to transform your morning routine? Here are 5 game-changing habits that successful entrepreneurs swear by!

    1. 🌅 Wake up at 5 AM - Own your morning, own your day
    2. 📚 Read for 30 ...

❌ OLD APPROACH (Generic):
------------------------------
Generic prompt focusing only on topic, not actual story
Result: Often produces generic 'success' or 'business' imagery

✅ NEW APPROACH (Content-Specific):
------------------------------
ANALYZING ACTUAL CONTENT FOR VISUAL ELEMENTS:
• Morning routine → sunrise, dawn imagery
• 5 specific habits → visual progression/steps
• Wake up at 5 AM → early morning/sunrise
• Reading → books/learning elements
• Exercise → movement/energy
• Meditation → calm/mindfulness
• Journaling → planning/goal-setting
• 30-day test → transformation journey
• 40% productivity inc

In [ ]:
# 📂 Improved File Organization Demo

def show_file_organization():
    """Demonstrate the new, cleaner file organization system"""
    
    print("📂 IMPROVED FILE ORGANIZATION")
    print("=" * 50)
    
    # Show directory structure
    print("📁 Directory Structure:")
    print("""
    posts/                      # Main content directory
    ├── images/                 # All generated images
    │   ├── instagram_230830_1145.png
    │   ├── linkedin_230830_1146.png
    │   └── ...
    ├── instagram_fitness_230830_1145.json
    ├── instagram_fitness_230830_1145.txt
    ├── linkedin_tech_230830_1146.json
    ├── linkedin_tech_230830_1146.txt
    └── ...
    """)
    
    print("🔍 File Naming Convention:")
    print("""
    Posts:
    {platform}_{niche}_{YYMMDD}_{HHMM}.{ext}
    Example: instagram_fitness_230830_1145.json
    
    Images:
    {platform}_{YYMMDD}_{HHMM}.png
    Example: instagram_230830_1145.png
    """)
    
    # List actual files
    print("\n📄 Current Files:")
    print("-" * 30)
    
    posts_dir = "posts"
    if os.path.exists(posts_dir):
        # List post files
        post_files = [f for f in os.listdir(posts_dir) if os.path.isfile(os.path.join(posts_dir, f))]
        if post_files:
            print("\nPost Files:")
            for f in sorted(post_files):
                print(f"• {f}")
        
        # List images
        images_dir = os.path.join(posts_dir, "images")
        if os.path.exists(images_dir):
            image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
            if image_files:
                print("\nImage Files:")
                for f in sorted(image_files):
                    print(f"• {f}")
    
    print("\n✨ Benefits:")
    print("✅ Cleaner, shorter filenames")
    print("✅ Organized by platform and content type")
    print("✅ Easy to find related files")
    print("✅ Consistent naming convention")
    print("✅ Separate image storage")
    print("✅ Better file management")
    
    print("\n💡 Usage Tips:")
    print("• Files are grouped by platform and niche")
    print("• Timestamps help track post history")
    print("• Images are stored separately but linked")
    print("• JSON for data, TXT for human reading")

# Show the new organization
show_file_organization()


📂 IMPROVED FILE ORGANIZATION
📁 Directory Structure:

    posts/                      # Main content directory
    ├── images/                 # All generated images
    │   ├── instagram_230830_1145.png
    │   ├── linkedin_230830_1146.png
    │   └── ...
    ├── instagram_fitness_230830_1145.json
    ├── instagram_fitness_230830_1145.txt
    ├── linkedin_tech_230830_1146.json
    ├── linkedin_tech_230830_1146.txt
    └── ...
    
🔍 File Naming Convention:

    Posts:
    {platform}_{niche}_{YYMMDD}_{HHMM}.{ext}
    Example: instagram_fitness_230830_1145.json

    Images:
    {platform}_{YYMMDD}_{HHMM}.png
    Example: instagram_230830_1145.png
    

📄 Current Files:
------------------------------

Post Files:
• instagram_lifestyle_250830_1233.json
• instagram_lifestyle_250830_1233.txt
• instagram_lifestyle_250831_1121.json
• instagram_lifestyle_250831_1121.txt
• instagram_lifestyle_250831_1132.json
• instagram_lifestyle_250831_1132.txt
• instagram_lifestyle_250831_1214.json
• instagra

# 🎨 Image Generation Workflow

## 🔄 Two-Step Process

### 1️⃣ Smart Prompt Generation (Gemini)
- Uses Google Gemini to analyze post content
- Creates detailed, content-specific prompts
- Ensures images match the story
- Generates alternative prompt variations

### 2️⃣ Actual Image Creation (DALL-E 3)
- Uses OpenAI's DALL-E 3 model
- Creates high-quality 1024x1024 images
- Saves in organized directory structure
- Platform-optimized output

## 📋 Example Workflow

1. **Content Analysis** (Gemini)
   ```python
   # Analyze post content
   content = "Morning routine transformation..."
   prompt = generate_smart_prompt(content)
   ```

2. **Prompt Generation** (Gemini)
   ```python
   # Create detailed image prompt
   "Create a visual journey showing 5 morning habits..."
   ```

3. **Image Creation** (DALL-E 3)
   ```python
   # Generate actual image
   image = dalle.generate(prompt)
   ```

4. **File Organization**
   ```
   posts/
   └── images/
       └── instagram_230830_1145.png
   ```

## 🎯 Why This Approach?

- **Gemini**: Excellent at understanding content and creating detailed prompts
- **DALL-E 3**: Best-in-class image generation
- **Combined**: Perfect images that match your content!


In [ ]:
# 🧪 Test Image Generation Workflow

def test_image_workflow():
    """Test the two-step image generation process"""
    
    print("🎨 TESTING IMAGE GENERATION WORKFLOW")
    print("=" * 60)
    
    # Sample content
    content = {
        'title': '5 Morning Habits for Success',
        'content': """
        Transform your mornings with these 5 powerful habits:
        1. 🌅 Wake up at 5 AM for a fresh start
        2. 🧘‍♂️ 15 minutes of mindful meditation
        3. 📝 Journal your goals and intentions
        4. 💪 Quick energizing workout
        5. 🥗 Nutritious breakfast prep
        
        I've tested this routine for 30 days and the results are amazing!
        My productivity is up 40% and my energy levels have never been better.
        """,
        'platform': 'instagram'
    }
    
    print("\n1️⃣ STEP 1: SMART PROMPT GENERATION (GEMINI)")
    print("-" * 40)
    
    # Generate prompt with Gemini
    prompt = image_gen.generate_image_prompt(
        {'title': content['title']}, 
        {'content': content['content'], 'platform': content['platform']}
    )
    
    print("\n2️⃣ STEP 2: IMAGE CREATION (DALL-E 3)")
    print("-" * 40)
    
    # Generate image with DALL-E 3
    image_result = image_gen.generate_actual_image(
        prompt,
        "test_workflow",
        content['platform']
    )
    
    if image_result.get('image_generated'):
        print("\n✅ SUCCESS!")
        print(f"📂 Image saved: {image_result['image_path']}")
        print(f"🔗 Image URL: {image_result['image_url']}")
    else:
        print("\n❌ Image generation failed")
        print(f"Error: {image_result.get('error', 'Unknown error')}")
    
    return image_result

# Run the test
print("🚀 Testing Image Generation Workflow...")
test_result = test_image_workflow()


🚀 Testing Image Generation Workflow...
🎨 TESTING IMAGE GENERATION WORKFLOW

1️⃣ STEP 1: SMART PROMPT GENERATION (GEMINI)
----------------------------------------
✅ Image prompt composed from content analysis

2️⃣ STEP 2: IMAGE CREATION (DALL-E 3)
----------------------------------------
✅ Placeholder image saved to: posts/images/instagram_250831_1236.png

✅ SUCCESS!
📂 Image saved: posts/images/instagram_250831_1236.png
🔗 Image URL: 
